In [16]:
import os
import csv
import sys
import random
import scipy
import numpy as np
import xgboost as xgb
# read in data


workspace_path = '/home/irashadow/python_workspace/Kaggle_Comp/Santander/'
month_dict_inv = dict()
month_dict_inv[0] = '2015-01'
month_dict_inv[1] = '2015-02'
month_dict_inv[2] = '2015-03'
month_dict_inv[3] = '2015-04'
month_dict_inv[4] = '2015-05'
month_dict_inv[5] = '2015-06'
month_dict_inv[6] = '2015-07'
month_dict_inv[7] = '2015-08'
month_dict_inv[8] = '2015-09'
month_dict_inv[9] = '2015-10'
month_dict_inv[10] = '2015-11'
month_dict_inv[11] = '2015-12'
month_dict_inv[12] = '2016-01'
month_dict_inv[13] = '2016-02'
month_dict_inv[14] = '2016-03'
month_dict_inv[15] = '2016-04'
month_dict_inv[16] = '2016-05'
month_dict_inv[17] = '2016-06'

In [157]:
cust_2_labels_dict = dict()

pos_case_dict = dict()
neg_case_dict = dict()


model_eval = 23


train_write = open(workspace_path + '/train/model_val/cust_product_m'+str(model_eval)+'_xgb.train', 'w')

valid_write = open(workspace_path + '/train/model_val/cust_product_m'+str(model_eval)+'_xgb.valid', 'w')

check_line_count = 0   
#data 5[2016June] for model #18
#data 16[2016May] for model #13
#data 12[2016Jan] for model #20
#data 11[2015Dec] for model #3
#data 9[2015Oct] for model #3
#----------------------------------
#try data 12[2016Jan] for model #5
#----------------------------------

for date_index_major in [5,15]:
    print 'Data:'+str(month_dict_inv[date_index_major])+' in processing..'
    
    feat_file = open(workspace_path + '/L4_feat_max/4_7_cust_date_nominal_prod_L4_'+str(month_dict_inv[date_index_major])+'_Model.feat', 'r')
    
    for line in feat_file:

        line = line.strip()
        check_line_count += 1

        if len(line.split('|')) < 6:

           print line 

        cust_id = line.split('|')[0]
        date_note = line.split('|')[1]
        label_tokens = line.split('|')[2]
        
        get_label = np.zeros(30)

        for label_item in label_tokens.split(','):

            if len(label_item) > 0:

                prod_id = int(label_item)
                get_label[prod_id] = 1

#-------
        if date_index_major in [5]:

           for j in range(1,25):
               
               if j not in [1,2,10,11,12,21]: 
                
                  if get_label[j] == 1 and j == model_eval:
 
                     prt_line = str(1) + ' ' 
                     for fi in range(3,9):
                         if len(line.split('|')[fi]) > 0:
                            prt_line += line.split('|')[fi] + ' ' 

                     train_write.write(prt_line[:-1]+'\n') 
                    
                  elif get_label[j] == 1 and j != model_eval:
 
                     prt_line = str(0) + ' ' 
                     for fi in range(3,9):
                         if len(line.split('|')[fi]) > 0:
                            prt_line += line.split('|')[fi] + ' ' 

                     train_write.write(prt_line[:-1]+'\n')   
                    
           
#-------
        if date_index_major in [15]:

           for j in range(1,25):
               
               if j not in [1,2,10,11,12,21]: 
                
                  if get_label[j] == 1 and j == model_eval:
 
                     prt_line = str(1) + ' ' 
                     for fi in range(3,9):
                         if len(line.split('|')[fi]) > 0:
                            prt_line += line.split('|')[fi] + ' ' 

                     valid_write.write(prt_line[:-1]+'\n') 
                    
                  elif get_label[j] == 1 and j != model_eval:
 
                     prt_line = str(0) + ' ' 
                     for fi in range(3,9):
                         if len(line.split('|')[fi]) > 0:
                            prt_line += line.split('|')[fi] + ' ' 

                     valid_write.write(prt_line[:-1]+'\n') 
                                         


    feat_file.close()           
 
train_write.close()  
valid_write.close()
print check_line_count    

Data:2015-06 in processing..
Data:2016-04 in processing..
65417


In [158]:
train_data_path = './train/model_val/cust_product_m'+str(model_eval)+'_xgb.train'

dtrain = xgb.DMatrix(train_data_path)

# specify parameters via map
param ={"objective" : "multi:softprob",    # multiclass classification 
              "num_class" : 2,    # number of classes 
              "eval_metric" : "mlogloss",    # evaluation metric 
              "nthread" : 6,   # number of threads to be used 
              "max_depth" : 14,    # maximum depth of tree 
              "eta" : 0.5,    # step size shrinkage 
              "subsample" : 0.8,    # part of data instances to grow tree 
              "colsample_bytree" : 0.8  # subsample ratio of columns when constructing each tree 
       }

num_round = 50
bst = xgb.train(param, dtrain, num_round)
# make prediction
preds = bst.predict(dtrain)

print preds.shape

(44055, 2)


In [159]:
preds_sort = np.argsort(preds, axis=1)
print preds_sort.shape
preds_top7 = np.fliplr(preds_sort)[:,:7]
print preds_top7

#----------------------------------------------------
dtrain_label = dtrain.get_label().astype(int)

print dtrain_label.shape[0]

label_train_dict = dict()
for i in range(0,dtrain_label.shape[0]):
#for i in range(0,100):
    
    top_product = dtrain_label.astype(int)[i]
    #print top_product
    
    if top_product not in label_train_dict:
       label_train_dict[top_product]  = 1
    else:
       label_train_dict[top_product]  += 1
    
print len(label_train_dict)    

(44055, 2)
[[0 1]
 [0 1]
 [1 0]
 ..., 
 [0 1]
 [0 1]
 [0 1]]
44055
2


In [160]:
print 'train distribution'

for item in label_train_dict:
    
    print 'label #' + str(item) + ' :%.2f'%(label_train_dict[item]/float(dtrain_label.shape[0])*100.0) 
    #print 'label #' + str(item)


train distribution
label #0 :81.32
label #1 :18.68


In [161]:
correct_count = 0
TP_count = 0
FP_count = 0
TN_count = 0
FN_count = 0

for i in range(0,len(dtrain_label)):
    
    if preds_top7[i,0] == dtrain_label[i]:

       correct_count += 1 
    
       if preds_top7[i,0] == 1 and dtrain_label[i] == 1:
            
          TP_count += 1  
    
       elif preds_top7[i,0] == 0 and dtrain_label[i] == 0:
            
          TN_count += 1  
    
    else:
        
       if preds_top7[i,0] == 1 and dtrain_label[i] == 0:
            
          FP_count += 1  
    
       elif preds_top7[i,0] == 0 and dtrain_label[i] == 1:
            
          FN_count += 1   
 

print 'accuracy:%.3f'%(correct_count/float(len(dtrain_label)))

print 'precision:%.3f'%(TP_count/float(TP_count+FP_count))
print 'recall:%.3f'%(TP_count/float(TP_count+FN_count))

print 'TP_count:'+str(TP_count)
print 'FP_count:'+str(FP_count)
print 'FN_count:'+str(FN_count)
print 'TN_count:'+str(TN_count)

accuracy:0.875
precision:0.721
recall:0.536
TP_count:4414
FP_count:1706
FN_count:3815
TN_count:34120


In [162]:
valid_data_path = './train/model_val/cust_product_m'+str(model_eval)+'_xgb.valid'
dvalid = xgb.DMatrix(valid_data_path)
dvalid_label = dvalid.get_label().astype(int)
preds = bst.predict(dvalid)



In [163]:
preds_sort = np.argsort(preds, axis=1)
print preds_sort.shape
preds_top7 = np.fliplr(preds_sort)[:,:15]
print preds_top7

label_submit_dict = dict()
for i in range(0,preds_sort.shape[0]):
#for i in range(0,100):
    
    top_product = preds_top7[i,0]
    
    if top_product not in label_submit_dict:
       label_submit_dict[top_product]  = 1
    else:
       label_submit_dict[top_product]  += 1
    

(35125, 2)
[[0 1]
 [0 1]
 [0 1]
 ..., 
 [0 1]
 [0 1]
 [0 1]]


In [164]:
print 'validation distribution'

for item in label_submit_dict:
    
    print 'label #' + str(item+1) + ' :%.2f'%(label_submit_dict[item]/float(preds_sort.shape[0])*100.0) 
    #print 'label #' + str(item)


validation distribution
label #1 :94.56
label #2 :5.44


In [165]:
correct_count = 0
TP_count = 0
FP_count = 0
TN_count = 0
FN_count = 0

for i in range(0,len(dvalid_label)):
    
    if preds_top7[i,0] == dvalid_label[i]:

       correct_count += 1 
    
       if preds_top7[i,0] == 1 and dvalid_label[i] == 1:
            
          TP_count += 1  
    
       elif preds_top7[i,0] == 0 and dvalid_label[i] == 0:
            
          TN_count += 1  
    
    else:
        
       if preds_top7[i,0] == 1 and dvalid_label[i] == 0:
            
          FP_count += 1  
    
       elif preds_top7[i,0] == 0 and dvalid_label[i] == 1:
            
          FN_count += 1   
 

print 'accuracy:%.3f'%(correct_count/float(len(dvalid_label)))

print 'precision:%.3f'%(TP_count/float(TP_count+FP_count))
print 'recall:%.3f'%(TP_count/float(TP_count+FN_count))

print 'TP_count:'+str(TP_count)
print 'FP_count:'+str(FP_count)
print 'FN_count:'+str(FN_count)
print 'TN_count:'+str(TN_count)

accuracy:0.884
precision:0.602
recall:0.257
TP_count:1151
FP_count:761
FN_count:3323
TN_count:29890


In [12]:
test_data_path = './test/5_4_cust_date_prod_L4_max.test'
dtest = xgb.DMatrix(test_data_path)
preds = bst.predict(dtest)

In [13]:
label_dict = dict()
label_dict[1] = 'ind_ahor_fin_ult1'
label_dict[2] = 'ind_aval_fin_ult1'
label_dict[3] = 'ind_cco_fin_ult1'
label_dict[4] = 'ind_cder_fin_ult1'
label_dict[5] = 'ind_cno_fin_ult1'
label_dict[6] = 'ind_ctju_fin_ult1'
label_dict[7] = 'ind_ctma_fin_ult1'
label_dict[8] = 'ind_ctop_fin_ult1'
label_dict[9] = 'ind_ctpp_fin_ult1'
label_dict[10] = 'ind_deco_fin_ult1'
label_dict[11] = 'ind_deme_fin_ult1'
label_dict[12] = 'ind_dela_fin_ult1'
label_dict[13] = 'ind_ecue_fin_ult1'
label_dict[14] = 'ind_fond_fin_ult1'
label_dict[15] = 'ind_hip_fin_ult1'
label_dict[16] = 'ind_plan_fin_ult1'
label_dict[17] = 'ind_pres_fin_ult1'
label_dict[18] = 'ind_reca_fin_ult1'
label_dict[19] = 'ind_tjcr_fin_ult1'
label_dict[20] = 'ind_valo_fin_ult1'
label_dict[21] = 'ind_viv_fin_ult1'
label_dict[22] = 'ind_nomina_ult1'
label_dict[23] = 'ind_nom_pens_ult1'
label_dict[24] = 'ind_recibo_ult1'

In [14]:
preds_sort = np.argsort(preds, axis=1)
print preds_sort.shape
preds_top7 = np.fliplr(preds_sort)[:,:7]
print preds_top7.shape

label_submit_dict = dict()
for i in range(0,preds_sort.shape[0]):
#for i in range(0,100):
    
    top_product = preds_top7[i,0]
    
    if top_product not in label_submit_dict:
       label_submit_dict[top_product]  = 1
    else:
       label_submit_dict[top_product]  += 1

    

(929615, 2)
(929615, 2)


In [15]:
print 'submission distribution'

for item in label_submit_dict:
    
    print 'label #' + str(item) + ' :%.2f'%(label_submit_dict[item]/float(preds_sort.shape[0])*100.0) 
    #print 'label #' + str(item)


submission distribution
label #0 :96.35
label #1 :3.65


In [74]:
Eout_write = open('./output/TheLastFourRecord_pred_Dec08_xgb_m18.eout', 'w')  

feat_data_path = workspace_path + '/input/test_ver2.csv'

test_read = open(feat_data_path, 'r')
headline = test_read.readline().strip()

line_count = 0
for line in test_read:
    
    line = line.strip().split(',')
    cust_id = int(line[1].replace(' ',''))
    
    prt_line_temp = ''
    
    
    product_index = preds_top7[line_count,0]
    
    if product_index == 1:
    
       prt_line = str(cust_id) + '|' + label_dict[18]  + '\n'
    
    else:
        
       prt_line = str(cust_id) + '|\n' 
    
    line_count += 1
    
    Eout_write.write(prt_line)
   
print line_count 
Eout_write.close()

929615


In [43]:
dict_read = open(workspace_path + '/dict/cust_last_prods.txt', 'r')

cust_end_products_dict = dict()

line_count = 0
for line in dict_read:  
    line = line.strip()
    
    line_count += 1
    
    cust_id = int(line.split('|')[0])
    last_prod_date = line.split('|')[1]
    label_tokens_get = line.split('|')[2]

    
    if cust_id not in cust_end_products_dict:
       cust_end_products_dict[cust_id] = [] 
    
    if len(label_tokens_get) > 0 and last_prod_date == '2016-05': 
    
        for feat_item in label_tokens_get.split(','):
            
            prod_id = int(feat_item)
            cust_end_products_dict[cust_id].append(label_dict[prod_id])
        
print line_count 
dict_read.close()

956645


In [44]:
Eout_dict_read = open('./output/TheLastFourRecord_pred_Dec08_xgb_m18.eout', 'r')  
id_2_prod_dict = dict()

for line in Eout_dict_read:
    line = line.strip()
    
    if len(line.split('|')) != 2:
       print 'error_line:'+line
    else:
       cust_id  = int(line.split('|')[0])
       products = line.split('|')[1]
        
       id_2_prod_dict[cust_id] = products 

    
sample_submission_read = open('./output/sample_submission.csv', 'r')

head_line = 'ncodpers,added_products' + '\n'
submission_write = open('./output/submission_Dec08-2_m8.csv', 'w')  
submission_write.write(head_line)

check_line_count = 0

duplicated_items_count = 0
#for i in range(0,10000):    
    #line = sample_submission_read.readline().strip()
    
for line in sample_submission_read:
    line = line.strip()    
    
    if 'ncodpers' not in line:
    
        cust_id  = int(line.split(',')[0])

        prt_line = str(cust_id) + ','
        
        pruned_line = False
        
        added_items_count = 0

        if cust_id in id_2_prod_dict:
            
           
           #print cust_end_products_dict[cust_id]
           #print 'predict:'+ str(id_2_prod_dict[cust_id])  
           
           for prod_name_item in id_2_prod_dict[cust_id].split(' '):
            
               
               if cust_id in cust_end_products_dict:
                   if prod_name_item not in cust_end_products_dict[cust_id]:
                        
                         if added_items_count < 7:
                            
                            prt_line += prod_name_item + ' '
                        
                            added_items_count += 1

                            pruned_line = True

                   else: 

                     #print '->' + prod_name_item    

                        duplicated_items_count += 1

        check_line_count += 1
        
        if pruned_line:
           submission_write.write(prt_line[:-1]+'\n')
        else:
           submission_write.write(prt_line+'\n')  

print check_line_count 
print duplicated_items_count
    
submission_write.close() 
sample_submission_read.close() 
Eout_dict_read.close() 

929615
2
